## import modules

In [9]:
from tqdm import tqdm
import re
import torch
import os
import random

import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

In [2]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # 디버깅 위한 세팅
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

print('Device:', torch.cuda.device)  # 출력결과: cuda 
print('Count of using GPUs:', torch.cuda.device_count())   
print('Current cuda device:', torch.cuda.current_device()) 

Device: <class 'torch.cuda.device'>
Count of using GPUs: 2
Current cuda device: 0


### Load Data
Data가 위치한 PATH에서 Data를 불러오기

In [3]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)

    return j

# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

In [4]:
train_data = jsonlload('nikluge-sa-2022-train.jsonl')
dev_data = jsonlload('nikluge-sa-2022-dev.jsonl')

## Preprocessing method

In [5]:
def preprocess(text: str, only_kor: bool = True):
    """한국어 문장을 옵션에 맞게 전처리"""
    # 한국어 모음과 특수 문자, 숫자 및 영어 제거
    if only_kor:
        text = re.sub(f"[^가-힣| |]+", "", text)
    else:
        text = re.sub(f"[^가-힣|ㄱ-ㅎ|0-9|]+", "", text)

    # 연속 공백 제거
    text = re.sub(" +", " ", text)

    # 좌우 불필요한 공백 제거
    return text.strip()

## 재현성 위한 seed 고정

In [10]:
def seed_everything(seed:int = 1004):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
    
seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## 태그셋 정의

In [14]:
#개체#속성 태그셋 정의
entity_property_pair = [
    '제품 전체#품질', '제품 전체#편의성', '제품 전체#디자인', '제품 전체#일반', '제품 전체#가격', 
    '제품 전체#인지도',  '제품 전체#다양성',
    '패키지/구성품#디자인', '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#일반',
    '패키지/구성품#편의성', '패키지/구성품#품질',
    '본품#일반', '본품#다양성', '본품#품질', '본품#인지도', '본품#편의성', '본품#디자인', '본품#가격',
    '브랜드#인지도', '브랜드#일반', '브랜드#디자인', '브랜드#품질', '브랜드#가격']

rep_entity_property_pair = [
    '제품전체#품질', '제품전체#편의성', '제품전체#디자인', '제품전체#일반', '제품전체#가격', 
    '제품전체#인지도',  '제품전체#다양성',
    '패키지/구성품#디자인', '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#일반',
    '패키지/구성품#편의성', '패키지/구성품#품질',
    '본품#일반', '본품#다양성', '본품#품질', '본품#인지도', '본품#편의성', '본품#디자인', '본품#가격',
    '브랜드#인지도', '브랜드#일반', '브랜드#디자인', '브랜드#품질', '브랜드#가격']

len(entity_property_pair)

25

## Load Tokenizer

In [17]:
MODEL_NAME = 'klue/roberta-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer

PreTrainedTokenizerFast(name_or_path='klue/roberta-large', vocab_size=32000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [18]:
special_tokens_dict = {
    'additional_special_tokens': rep_entity_property_pair
}

print(tokenizer.vocab_size)

# 토크나이저는 각 모델 별로 만들지 않아도 된다. [임베딩 layer에만 영향 주므로]
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

print(num_added_toks)

32000
25


In [19]:
print(tokenizer.tokenize("야 아이패드 가격 왜 이래 본품#가격"))
print(tokenizer("야 아이패드 가격 왜 이래 본품#가격"))
print(tokenizer.decode(tokenizer.encode("야 아이패드 가격 왜 이래 본품#가격")))

['야', '아이패드', '가격', '왜', '이래', '본품#가격']
{'input_ids': [0, 1396, 15641, 3852, 1460, 5625, 32019, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] 야 아이패드 가격 왜 이래 본품#가격 [SEP]


## Load Model

In [21]:
ACD_set = []

for i in range(len(rep_entity_property_pair)):
    tmp_model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)  #True/False 
    tmp_model.resize_token_embeddings(tokenizer.vocab_size + num_added_toks)
    ACD_set.append(tmp_model)

ACD_set[7]

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifi

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the

Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifi

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32025, 1024)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm